In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [31]:
# 1. Caricamento del file sorgente
SOURCE_FILE = r"EntryAccessoAmministrati_202501.csv"

df = pd.read_csv(SOURCE_FILE)

# 2. Mappatura (IT ➜ EN) dei nomi colonna
col_map = {
    "regione_residenza_domicilio": "region_of_residence",
    "amministrazione_appartenenza": "administration",
    "sesso": "gender",
    "eta_max": "age_max",
    "eta_min": "age_min",
    "modalita_autenticazione": "Access_method",
    "numero_occorrenze": "employee_count",
}

df = df.rename(columns=col_map)

# 3. Pulizia / conversione dei tipi
#    ▸ eventuali stringhe vuote in 'age_max' diventano 0
#    ▸ conversione a interi
df["age_max"] = (
    pd.to_numeric(df["age_max"], errors="coerce")  # '34', ' ' … ➜ numerico / NaN
    .fillna(0)                                     # NaN ➜ 0
    .astype(int)
)
df["age_min"] = df["age_min"].astype(int)

# 4. (Facoltativo) Ordine coerente delle colonne
df = df[list(col_map.values())]

output_file = r"Accesso_English1.csv"
df.to_csv(output_file, index=False)


In [7]:
# 1. File I/O
SOURCE_FILE = r"EntryAccreditoStipendi_202501.csv"

df = pd.read_csv(SOURCE_FILE)

# 2. Mapping IT ➜ EN delle colonne
col_map = {
    "comune_della_sede":  "office_municipality",
    "amministrazione":    "administration",
    "eta_min":            "age_min",
    "eta_max":            "age_max",
    "sesso":              "gender",
    "modalita_pagamento": "payment_method",
    "numero":             "salary_count",
}

df = df.rename(columns=col_map)

# 3. Pulizia / conversione dei tipi
df["age_max"] = (
    pd.to_numeric(df["age_max"], errors="coerce")  # es. ' ' ➜ NaN
      .fillna(0)                                   # NaN ➜ 0
      .astype(int)
)
df["age_min"]    = df["age_min"].astype(int)
df["salary_count"] = df["salary_count"].astype(int)

# 4. (Facoltativo) Ordina le colonne come nel file di destinazione
df = df[list(col_map.values())]




In [ ]:
output_file_stipendi = r"Stipendi_English.csv"
df.to_csv(output_file_stipendi, index=False)

In [ ]:
import re
# 1. File I/O
SOURCE_FILE = r"EntryAmministratiPerFasciaDiReddito_202501.csv"

df = pd.read_csv(SOURCE_FILE)

# 2. Mappatura IT ➜ EN delle colonne
col_map = {
    "comparto":              "sector",
    "regione_residenza":     "region_of_residence",
    "sesso":                 "gender",
    "eta_min":               "age_min",
    "eta_max":               "age_max",
    "aliquota_max":          "max_tax_rate",
    "fascia_reddito_min":    "income_bracket_min",
    "fascia_reddito_max":    "income_bracket_max",
    "numerosita":            "employee_count",
}
df = df.rename(columns=col_map)

# 3. Funzione di supporto: estrae la prima cifra da stringhe tipo
#    "Fino a 28.000" o "Oltre i 28000"
def extract_number(txt: str | float | int):
    if pd.isna(txt):
        return None
    m = re.search(r"(\d[\d\.]*)", str(txt))
    if not m:
        return None
    return int(m.group(1).replace(".", ""))

# 4. Parsing delle fasce di reddito
df["income_bracket_min"] = df["income_bracket_min"].apply(extract_number)
df["income_bracket_max"] = df["income_bracket_max"].apply(extract_number)

#    ▸ se il minimo è mancante (riga con "Fino a …") lo impostiamo a 0
df["income_bracket_min"] = df["income_bracket_min"].fillna(0).astype(int)

# 5. Conversione e pulizia dei tipi numerici
num_cols = ["age_min", "age_max", "max_tax_rate", "income_bracket_max", "employee_count"]
for c in num_cols:
    df[c] = pd.to_numeric(df[c], errors="coerce")

df["income_bracket_max"] = df["income_bracket_max"].astype("Int64")   # può restare NaN
df["employee_count"] = df["employee_count"].fillna(0).astype(int)

# 6. (Facoltativo) Ordina le colonne come nel file di destinazione
df = df[list(col_map.values())]


In [ ]:
output_file_reddito = r"Reddito_English.csv"
df.to_csv(output_file_reddito, index=False)

In [27]:
# 1. File di origine/destinazione
SOURCE_FILE = r"EntryPendolarismo_202501.csv"

df = pd.read_csv(SOURCE_FILE)

# 2. Mapping IT ➜ EN dei nomi colonna
col_map = {
    "provincia_della_sede": "province_of_office",
    "comune_della_sede":    "office_municipality",
    "stesso_comune":        "same_municipality",
    "ente":                 "organization",
    "numero_amministrati":  "employee_count",
    "distance_min_KM":      "commuting_distance_min_km",
    "distance_max_KM":      "commuting_distance_max_km",
}

df = df.rename(columns=col_map)

# 3. Conversioni e pulizia
#    ▸ same_municipality: "SI"/"NO" ➜ boolean
df["same_municipality"] = df["same_municipality"].str.upper().eq("SI")

#    ▸ distanza min/max ➜ interi
for c in ["commuting_distance_min_km", "commuting_distance_max_km"]:
    df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

#    ▸ employee_count ➜ intero
df["employee_count"] = pd.to_numeric(df["employee_count"], errors="coerce").fillna(0).astype(int)

# 4. nuova colonna: average_commuting_distance
df["average_commuting_distance"] = (
    (df["commuting_distance_min_km"] + df["commuting_distance_max_km"]) / 2
).round().astype(int)

# 5. (Opzionale) Ordina le colonne come nel file di destinazione
ordered_cols = list(col_map.values()) + ["average_commuting_distance"]
df = df[ordered_cols]

In [28]:
df.head()

,province_of_office,office_municipality,same_municipality,organization,employee_count,commuting_distance_min_km,commuting_distance_max_km,average_commuting_distance
0,CATANIA,ACIREALE,False,CONSIGLIO PER RIC. IN AGRICOLTURA E ANALISI EC...,11,5,10,8
1,VITERBO,ACQUAPENDENTE,False,MINISTERO DELL'ISTRUZIONE E DEL MERITO,8,200,600,400
2,VITERBO,ACQUAPENDENTE,False,MINISTERO DELL'ISTRUZIONE E DEL MERITO,17,50,100,75
3,NAPOLI,AFRAGOLA,False,MINISTERO DELL'ISTRUZIONE E DEL MERITO,371,20,50,35
4,PESCARA,ALANNO,False,MINISTERO DELL'ISTRUZIONE E DEL MERITO,27,5,10,8


In [23]:
output_file_pendolarismo = r"Pendolarismo_English.csv"
df.to_csv(output_file_pendolarismo, index=False)